In [1]:
#Accidentalidad Madrid part 2

In [3]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

In [4]:
df=pd.read_csv('df_part1.csv')
test=pd.read_csv('test_part1.csv')

## Arranging the data

In [23]:
# now we are going to extract the percentage of accidents per district
aux=df.groupby(['distrito'])['dow'].count().reset_index().rename(columns={'dow': "num_accidentes"})
aux['peso_distrito']=aux['num_accidentes']*100/aux['num_accidentes'].sum()

# now we merge these values into df and test
df1=pd.merge(df,aux[['distrito','peso_distrito']], on='distrito')
test1=pd.merge(test, aux[['distrito','peso_distrito']], on='distrito')

In [26]:
df1.isnull().sum()

dow                         0
time_int                    0
distrito                    0
tipo_accidente              0
tipo_persona                0
rango_edad                  0
sexo                        0
cod_lesividad           14802
lesividad               14802
coordenada_x_utm            2
coordenada_y_utm            2
positiva_alcohol          102
num_implicados              0
despejado_checked           0
lluviad_checked             0
lluviai_checked             0
nieve_checked               0
granizo_checked             0
nublado_nchecked            0
desconocido_nchecked        0
cruce                       0
vehi__ciclomotor            0
vehi__furgoneta             0
vehi__resto                 0
vehi__turismo               0
peso_distrito               0
dtype: int64

In [38]:
#We are going to get the percentage of accidents by time slot (low, high, night) and by age and sex
aux=df1.groupby(['rango_edad']).count().reset_index().iloc[:,0:2]
aux['high']=

,rango_edad,dow
0,De 10 a 14 años,305
1,De 15 a 17 años,251
2,De 18 a 20 años,978
3,De 21 a 24 años,2230
4,De 25 a 29 años,3444
5,De 30 a 34 años,3364
6,De 35 a 39 años,3336
7,De 40 a 44 años,3406
8,De 45 a 49 años,3094
9,De 50 a 54 años,2543


In [45]:
aux=df1.groupby(["rango_edad","time_int"]).count().iloc[:,1:2]
aux

distrito
rango_edad      time_int          
De 10 a 14 años high            58
                low            235
                night           12
De 15 a 17 años high            45
                low            176
                night           30
De 18 a 20 años high           137
                low            665
                night          176
De 21 a 24 años high           290
                low           1565
                night          375
De 25 a 29 años high           535
                low           2493
                night          416
De 30 a 34 años high           562
                low           2484
                night          318
De 35 a 39 años high           627
                low           2401
                night          308
De 40 a 44 años high           618
                low           2517
                night          271
De 45 a 49 años high           633
                low           2243
                night          218
De 50 a 54 años high           492
                low           1867
                night          184
De 55 a 59 años high           400
                low           1537
                night          143
De 6 a 9 años   high            40
                low            133
                night            1
De 60 a 64 años high           255
                low            955
                night           64
De 65 a 69 años high           100
                low            522
                night           18
De 70 a 74 años high            68
                low            350
                night           11
Desconocido     high           528
                low           2474
                night          932
Menor de 5 años high            60
                low            223
                night            8
Más de 74 años  high            98
                low            553
                night            7

In [50]:
lista_edad=df1['rango_edad'].unique()
grupoe=[]
denoche=[]
depunta=[]
devalle=[]
for i in lista_edad:
    grupoe.append(i)
    edad=df1[df1['rango_edad']==i]
    noche=edad[edad['time_int']=='night']
    denoche.append(noche['dow'].count())
    punta=edad[edad['time_int']=='high']
    depunta.append(punta['dow'].count())
    valle=edad[edad['time_int']=='low']
    devalle.append(valle['dow'].count())
    
rang_ed_horas=pd.DataFrame(list(zip(grupoe,denoche,depunta,devalle)),columns=['rango_edad','night','high','low'])
rang_ed_horas=rang_ed_horas.sort_values('rango_edad',ascending=True)
rang_ed_horas=rang_ed_horas.reset_index(drop=True)
rang_ed_horas

,rango_edad,night,high,low
0,De 10 a 14 años,12,58,235
1,De 15 a 17 años,30,45,176
2,De 18 a 20 años,176,137,665
3,De 21 a 24 años,375,290,1565
4,De 25 a 29 años,416,535,2493
5,De 30 a 34 años,318,562,2484
6,De 35 a 39 años,308,627,2401
7,De 40 a 44 años,271,618,2517
8,De 45 a 49 años,218,633,2243
9,De 50 a 54 años,184,492,1867
